In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None


In [ ]:
df=pd.read_csv('https://drive.google.com/uc?export=download&id=12KieiuP24jrrpTQqT4lj9L5iUXyr63s1')

In [ ]:
features_kept=['_STATE','SEXVAR','GENHLTH','PHYSHLTH','MENTHLTH','PRIMINSR','PERSDOC3','CHECKUP1','EXERANY2','SLEPTIM1','CVDINFR4','CHCOCNC1','ADDEPEV3','CHCKDNY2','DIABETE4','MARITAL','VETERAN3','WEIGHT2','HEIGHT3','DEAF','BLIND','DECIDE','DIFFWALK','DIFFDRES','DIFFALON','COVIDPOS','CNCRDIFF','CNCRAGE','CNCRTYP2','CSRVPAIN','CIMEMLOS','CDHOUSE','CDASSIST','CDHELP','CDSOCIAL','LSATISFY','EMTSUPRT','SDHISOLT','SDHEMPLY','SDHFOOD1','SDHBILLS','SDHUTILS','SDHTRNSP','SDHSTRE1','_URBSTAT','_IMPRACE','_RFHLTH','_PHYS14D','_MENT14D','_TOTINDA','_MICHD','_ASTHMS1','_DRDXAR2','_AGE80','_CHLDCNT','_EDUCAG','_INCOMG1','_SMOKER3','_CURECI2','_DRNKWK2','_AIDTST4']
df2 = df[features_kept]

In [ ]:
df3 = df2[['_STATE','SEXVAR','GENHLTH','PHYSHLTH','MENTHLTH','PRIMINSR','PERSDOC3','CHECKUP1','EXERANY2','CVDINFR4']]

In [ ]:
def ToOrdinal(df, column_name, Never=8, Never_to_0=True):
  df2 = df.copy()
  df2[column_name] = df2[column_name].astype(int)
  if Never:
    if Never_to_0:
      df2[column_name].replace(Never, 0, inplace=True)
    else:
      Never_to = df2[column_name].value_counts().sort_index(ascending=False).index[0]+1
      df2[column_name].replace(Never, Never_to, inplace=True)

  return df2

def ToBinary(df, column_name):
  df2 = df.copy()
  df2[column_name] = df2[column_name].astype(int)
  df2[column_name].replace(2, 0, inplace=True)
  return df2

def ToNominal(df, column_name, categories):
  df2 = df.copy()
  column_keys = df2[column_name].value_counts().index.tolist()
  column_keys.sort()

  mapping = {column_keys[i]: categories[i] for i in range(len(column_keys))}
  df2[column_name] = df2[column_name].map(mapping)
  return df2


In [ ]:
def Impute(df, column_name, Unknown=7, Missing=9, Impute_based_on=None):
  df2 = df.copy()

  if Impute_based_on is None:
    mode = df2[column_name].mode()[0]
    df2[column_name].replace([Unknown, Missing], mode, inplace=True)

  else:
    NA_filter = df2[column_name].isin([Unknown, Missing])
    g = df2[~NA_filter].groupby(Impute_based_on)
    Mode = {group: g.get_group(group)[column_name].mode()[0] for group in g.groups}

    sub_na = df2[NA_filter]
    sub_na[column_name] = sub_na[Impute_based_on].map(Mode)
    df2.update(sub_na)

  return df2

def OneHotEncoding(df):
  df2 = df.copy()
  one_hot_columns = df2.dtypes[df2.dtypes == 'object'].index.tolist()
  for column in one_hot_columns:
    one_hot_encoding = pd.get_dummies(df2[column], dtype='int')
    df2 = pd.concat([df2, one_hot_encoding], axis=1)
    df2.drop(columns=[column], inplace = True)
    if 'Missing' in df2.columns:
      df2.drop(columns=['Missing'], inplace = True)

  return df2

In [ ]:
test = ToNominal(df3, '_STATE', ['Florida', 'Idaho', 'Indiana', 'Maine', 'Nevada', 'Oregon', 'Rhode Island',
                'South Carolina', 'Utah', 'Vermont', 'Virginia', 'Wisconsin'])
test = ToBinary(test, 'SEXVAR')
test = ToOrdinal(test, 'GENHLTH')
test = ToOrdinal(test, 'PHYSHLTH', Never=88, Never_to_0=True)
test = ToOrdinal(test, 'MENTHLTH', Never=88, Never_to_0=True)
test = ToNominal(test, 'PRIMINSR', ['Insur_Employer/Union', 'Insur_Private Plan', 'Insur_Medicare', 'Insur_Medigap',
                   'Insur_Medicaid','Insur_CHIP', 'Insur_Military', 'Insur_Indian', 'Insur_State',
                   'Insur_Gvmt', 'Missing', 'Insur_None', 'Missing'])
test = ToOrdinal(test, 'PERSDOC3', Never=3, Never_to_0=True)
test = ToOrdinal(test, 'CHECKUP1', Never=8, Never_to_0=False)
test = ToBinary(test, 'EXERANY2')
test = ToBinary(test, 'CVDINFR4')


In [ ]:
test = Impute(test, 'GENHLTH')
test = Impute(test, 'PHYSHLTH', Unknown=77, Missing=99, Impute_based_on='GENHLTH')
test = Impute(test, 'MENTHLTH', Unknown=77, Missing=99, Impute_based_on='GENHLTH')
test = Impute(test, 'PERSDOC3', Impute_based_on='PRIMINSR')
test = Impute(test, 'CHECKUP1', Impute_based_on='PRIMINSR')
test = Impute(test, 'CVDINFR4', Impute_based_on='PHYSHLTH')
test = Impute(test, 'EXERANY2', Impute_based_on='GENHLTH')

In [ ]:
test = OneHotEncoding(test)

In [ ]:
test.to_csv('X_Xinya.csv', index=False)

In [ ]:
# Mappings:
# SEXVAR: {0: 'Male', 1: 'Female'}
# GENHLTH: General Health Status? {0: 'Unknown', 1: 'Excellent', 2: 'Very Good', 3: 'Good', 4: 'Fair', 5: 'Poor'}
# PHYSHLTH, MENTHLTH: Number of Days Physical/Mental Health Not Good? 0-30: Number of days
# PERSDOC3: Have Personal Health Care Provider? {0: 'No', 1: 'Only one', 2: 'More than one'}
# CHECKUP1: Length of time since last routine checkup? {0: 'Unknown', 1: 'Within past year', 2: 'Within past 2 years',
#                                                       3: 'Within past 5 years', 4: '5 or more years ago', 5: 'Never'}
# EXERANY2: Exercise in Past 30 Days? {0: 'No', 1: 'Yes'}
# CVDINFR4: Ever Diagnosed with Heart Attack? {0: 'No', 1: 'Yes'}

In [ ]:
y_columns=['CIMEMLOS','CDHOUSE','CDASSIST','CDHELP','CDSOCIAL']
output = df[y_columns]

In [ ]:
output.CIMEMLOS.value_counts()

CIMEMLOS
2.0    56945
1.0     7003
7.0      474
9.0      253
Name: count, dtype: int64

In [ ]:
refused_idx = output['CIMEMLOS'] == 9
remaining_output = output[~refused_idx]

In [ ]:
remaining_output.CIMEMLOS.replace(7, 1, inplace=True)
remaining_output.CIMEMLOS.replace(2, 0, inplace=True)

In [ ]:
mapping = {1: 'Always', 2: 'Usually', 3: 'Sometimes', 4: 'Rarely', 5: 'Never', 7: 'Never', 9: 'Never', 0: 'Missing'}
for column in ['CDHOUSE','CDASSIST','CDHELP','CDSOCIAL']:
  remaining_output[column] = remaining_output[column].fillna(0)
  remaining_output[column] = remaining_output[column].map(mapping) # Replace Don't know/not sure and Refused with Never

In [ ]:
mapping2 = {'Always': 5, 'Usually': 4, 'Sometimes': 3, 'Rarely': 2, 'Never': 1, 'Missing': 1}
for column in ['CDHOUSE','CDASSIST','CDHELP','CDSOCIAL']:
  remaining_output[column] = remaining_output[column].map(mapping2)

In [ ]:
remaining_output['y_sum'] = remaining_output[['CDHOUSE','CDASSIST','CDHELP','CDSOCIAL']].sum(axis=1) * remaining_output['CIMEMLOS']

In [ ]:
remaining_output['y_binary'] = remaining_output['y_sum'] > 0
remaining_output.y_binary = remaining_output.y_binary.astype(int)

In [ ]:
remaining_output.drop(columns=['CIMEMLOS', 'CDHOUSE','CDASSIST','CDHELP','CDSOCIAL'], inplace=True)
remaining_output

,y_sum,y_binary
0,5.0,1
1,0.0,0
2,4.0,1
3,0.0,0
4,0.0,0
...,...,...
64670,0.0,0
64671,0.0,0
64672,0.0,0
64673,5.0,1
